In [ ]:
### 참고한웹사이트 http://baekse.me/Melon-Chart-Crawling/


from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re

result = list()


def getSongs():
    """song_ids와 ranks에서 id에 matching되는 정보와 rank를 json 파일로 반환"""
    
    for rank, song_id in zip(ranks, song_ids):
        sleep(5)
        print(song_id)

        req = requests.get('http://www.melon.com/song/detail.htm?songId=' + song_id, headers=header)
        html = req.text
        soup = BeautifulSoup(html, "html.parser")

        title = soup.find(attrs={"class": "song_name"}).text.replace('곡명', '')

        if '19금' in title:
            title = title.replace('19금', '')

        title = re.sub('^\s*|\s+$', '', title)

        artist = soup.find(attrs={"class": "artist_name"}).text

        album = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[0].text

        genre = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[2].text

        # lyric = re.sub('<[^>]*>|\s|\[|\]', ' ', str(soup.find_all(attrs={"class": "lyric"})[0]))
        # lyric = re.sub('^\s*|\s+$', '', lyric)

        result.append({
            'year': re.sub('[^0-9]', '', year.text),
            'month': re.sub('[^0-9]', '', month.text),
            'week': week.text,
            'rank': rank.text,
            'title': title,
            'artist': artist,
            'album': album,
            'genre': genre
        })
        print("차트 연도:", year.text)
        print("차트 월간:", month.text)
        print("차트 주간:", week.text)
        print("순위:", rank.text)
        print("곡 id:", song_id)
        print("제목:", title)
        print("아티스트:", artist)
        print("앨범:", album)
        print("장르:", genre)
        print("*_*_*_*_*_*_*_*_*_*_*_*_*_*")

        
# acess the website    
header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/jisun/Desktop/Yonsei/Yonsei_19_1/Data Science/chromedriver')

# 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다.
d.implicitly_wait(3)
d.get('http://www.melon.com/chart/index.htm')
d.get("http://www.melon.com/chart/search/index.htm")

d.find_element_by_css_selector('#d_chart_search > div > h4.tab01 > a').click()

# h4.tab01: 주간차트 / h4.tab02: 월간차트 / h4.tab03: 연도차트

# 연대선택
age_selector='#d_chart_search > div > div > div.box_chic.nth1.view > div.list_value > ul > li:nth-child(1) > span > label'

age = d.find_element_by_css_selector(age_selector)
age.click()

for i in [5]: # 6: 2013, 5: 2014 순서대로 (옛날부터)
    # 년도선택
    year_selector = '#d_chart_search > div > div > div.box_chic.nth2.view > div.list_value > ul > li:nth-child(' + str(i) + ') > span > label'


    
    year = d.find_element_by_css_selector(year_selector)
    year.click()
    
    for i in [12]:
        # 월선택
        month_selector = '#d_chart_search > div > div > div.box_chic.nth3.view> div.list_value > ul > li:nth-child(' + str(i) + ') > span > label'
        
        month = d.find_element_by_css_selector(month_selector)
        month.click()
        
        for i in [3,4,5]:
            # 주간선택
            try:
                week_selector = '#d_chart_search > div > div > div.box_chic.nth4.view > div.list_value > ul > li:nth-child(' + str(i) + ') > span > label'

                week = d.find_element_by_css_selector(week_selector)
                week.click()
            except:
                print("week_selector not found")
                continue
            
            # 차트 가요 선택: 11년도 12년도 국내종합 분류가 없음
            
            
            classCd = d.find_element_by_css_selector('#d_chart_search > div > div > div.box_chic.last.view > div.list_value > ul > li:nth-child(1) > span > label')
            if '가요' not in classCd.text:
                classCd = d.find_element_by_css_selector('#d_chart_search > div > div > div.box_chic.last.view > div.list_value > ul > li:nth-child(2) > span > label')
            classCd.click()
            
            d.find_element_by_css_selector('#d_srch_form > div.wrap_btn_serch > button > span > span').click()
            sleep(10)
            
            # get song ids and ranks for rank 1~50: song50_ids & ranks50
         
            song_ids = d.find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > a')
            song_ids = [re.sub('[^0-9]', '', song_id.get_attribute("href")) for song_id in song_ids]
            ranks = d.find_elements_by_css_selector('#lst50 > td:nth-child(2) > div > span.rank')
            
            getSongs()
            
            # access next page
            next = d.find_element_by_css_selector('#frm > div.paginate.chart_page > span > a') 
            next.click()
            
            # get song ids and ranks for rank 51~100: song100_ids & ranks100
            song_ids = d.find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > a')
            song_ids = [re.sub('[^0-9]', '', song_id.get_attribute("href")) for song_id in song_ids]
            ranks = d.find_elements_by_css_selector('#lst100 > td:nth-child(2) > div > span.rank')

            getSongs()

d.quit()

#print("THIS IS THE LIST, RESULT")
#print(result)

7975242
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 1
곡 id: 7975242
제목: 사랑이 맞을거야
아티스트: 윤미래
앨범: 사랑이 맞을거야
장르: 발라드
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7956304
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 2
곡 id: 7956304
제목: DADDY (Feat. CL of 2NE1)
아티스트: 싸이 (PSY)
앨범: 칠집싸이다
장르: 댄스
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7965449
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 3
곡 id: 7965449
제목: 유레카 (Feat. Zion.T)
아티스트: 지코 (ZICO)
앨범: GALLERY
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7956303
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 4
곡 id: 7956303
제목: 나팔바지
아티스트: 싸이 (PSY)
앨범: 칠집싸이다
장르: 댄스
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7971052
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 5
곡 id: 7971052
제목: Sing For You
아티스트: EXO
앨범: SING FOR YOU - 겨울 스페셜 앨범, 2015
장르: 발라드
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7911687
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 6
곡 id: 7911687
제목: Boys And Girls (Feat. Babylon)
아티스트: 지코 (ZICO)
앨범: Boys and Girls
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7933703
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20


7965448
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 54
곡 id: 7965448
제목: VENI VIDI VICI (Feat. DJ Wegun)
아티스트: 지코 (ZICO)
앨범: GALLERY
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7870705
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 55
곡 id: 7870705
제목: 리듬 타 (RHYTHM TA)
아티스트: iKON
앨범: WELCOME BACK
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7847861
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 56
곡 id: 7847861
제목: Dumb Dumb
아티스트: Red Velvet (레드벨벳)
앨범: The Red - The 1st Album
장르: 댄스
*_*_*_*_*_*_*_*_*_*_*_*_*_*
5835766
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 57
곡 id: 5835766
제목: Okey Dokey
아티스트: MINO (송민호)
앨범: 쇼미더머니 4 Episode 6
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7977139
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 58
곡 id: 7977139
제목: 12시 25분 (Wish List)
아티스트: f(x)
앨범: 12시 25분 (Wish List) - WINTER GARDEN
장르: 댄스, 일렉트로니카
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7888412
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.14~12.20
순위: 59
곡 id: 7888412
제목: 말해 Yes Or No (Feat. PENOMECO, The Quiett)
아티스트: 지코 (ZICO)
앨범: 말해 Ye

7956303
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 5
곡 id: 7956303
제목: 나팔바지
아티스트: 싸이 (PSY)
앨범: 칠집싸이다
장르: 댄스
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7971052
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 6
곡 id: 7971052
제목: Sing For You
아티스트: EXO
앨범: SING FOR YOU - 겨울 스페셜 앨범, 2015
장르: 발라드
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7911687
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 7
곡 id: 7911687
제목: Boys And Girls (Feat. Babylon)
아티스트: 지코 (ZICO)
앨범: Boys and Girls
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7861392
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 8
곡 id: 7861392
제목: 또 다시 사랑
아티스트: 임창정
앨범: 또 다시 사랑
장르: 발라드
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7933703
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 9
곡 id: 7933703
제목: 못먹는 감 (Sour Grapes)
아티스트: San E
앨범: 못먹는 감
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
3973781
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 10
곡 id: 3973781
제목: 크리스마스니까
아티스트: 성시경
앨범: Jelly Christmas 2012 HEART PROJECT
장르: 발라드
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7891119
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.2

418598
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 57
곡 id: 418598
제목: 친구라도 될 걸 그랬어
아티스트: 거미
앨범: Like Them
장르: R&B/Soul
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7896507
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 58
곡 id: 7896507
제목: Zeze
아티스트: 아이유
앨범: CHAT-SHIRE
장르: 댄스
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7977072
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 59
곡 id: 7977072
제목: 꿀
아티스트: 40
앨범: 꿀
장르: R&B/Soul
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7965450
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 60
곡 id: 7965450
제목: 오만과 편견 (Feat. 수란)
아티스트: 지코 (ZICO)
앨범: GALLERY
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
4543502
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 61
곡 id: 4543502
제목: 야생화
아티스트: 박효신
앨범: 야생화
장르: 발라드
*_*_*_*_*_*_*_*_*_*_*_*_*_*
5644620
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 62
곡 id: 5644620
제목: CALL ME BABY
아티스트: EXO
앨범: The 2nd Album `EXODUS`
장르: 댄스
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7963607
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.21~12.27
순위: 63
곡 id: 7963607
제목: 북두칠성
아티스트: 로이킴
앨범: 북두칠성
장르: 발라드
*_*_*_*_*_*_

7861392
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.28~01.03
순위: 10
곡 id: 7861392
제목: 또 다시 사랑
아티스트: 임창정
앨범: 또 다시 사랑
장르: 발라드
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7985328
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.28~01.03
순위: 11
곡 id: 7985328
제목: 다시 (Feat. 유재석)
아티스트: 터보
앨범: AGAIN (다시)
장르: 댄스
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7933703
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.28~01.03
순위: 12
곡 id: 7933703
제목: 못먹는 감 (Sour Grapes)
아티스트: San E
앨범: 못먹는 감
장르: 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7935396
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.28~01.03
순위: 13
곡 id: 7935396
제목: HOT PINK
아티스트: EXID
앨범: HOT PINK
장르: 댄스
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7953958
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.28~01.03
순위: 14
곡 id: 7953958
제목: 우주를 건너
아티스트: 백예린
앨범: FRANK
장르: R&B/Soul
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7991610
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.28~01.03
순위: 15
곡 id: 7991610
제목: 덤앤더머 (DUMB & DUMBER)
아티스트: iKON
앨범: WELCOME BACK
장르: 댄스, 랩/힙합
*_*_*_*_*_*_*_*_*_*_*_*_*_*
7896508
차트 연도: 2015년
차트 월간: 12월
차트 주간: 12.28~01.03
순위: 16
곡 id: 7896508
제목: 스물셋
아티스트: 아이유
앨범: CHAT-